# Verslag modelleren

*gemaakt door Kees Brouwer en Robbert van Nieuwkerk*

Eerst het importeren van de benodigde libraries

In [1]:
# importeren libs
import numpy as np
import pandas as pd
import datetime
from itertools import product
# import itertools
import pickle
import copy

# import sys
# import dovpanda
import Hulpfuncties
import CrewCombined

In [2]:
# print(sys.getsizeof(dfSprint1))

er moet ook nog een keer worden uitgelegd wat er in welk df staat en wat er met ieder df is gedaan voordat het geimporteerd is. !TODO

Het gegeven excelbestand wat de informatie vat, is op een aantal punten aangepast voordat het is geimporteerd. 
Het bestand heeft voor iedere skill een aparte kolom. Mocht de skill nodig zijn voor de gegeven opdracht, dan staat er een 'x' in deze kolom. Ditzelfde is gedaan met de skills van medewerkers, de dagen waarop de medewerkers werken en voor de kamers waarin de opdrachten moeten worden uitgevoerd. Voor de additional tasks zijn er een aantal opdracht die in iedere sprint moeten worden uitgevoerd. Deze zijn opgesplitst, voor iedere sprint een aparte lijn. Hierbij is ook een kolom toegevoegd met het aantal frequenties voor deze opdracht. Ook is er een kolom toegevoegd bij het tabblad 'additional tasks' waarin staat aangegeven met hoeveel personen het minimaal moet worden uitgevoerd.
Hieronder worden de verschillende tabbladen ingelezen en toegekent aan een dataframe.
Verdere bewerkingen op de data worden gedaan in dit document.


In [3]:
# importeren data
dfProjectTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Project tasks')
dfAdditionalTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Additional tasks')
dfCrew = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Crew')
dfRooms = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Rooms')
dfHiring = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Hiring')

Het schoonmaken van de cel, hierdoor kunnen we waardes makkelijker worden teruggevonden dan NaN

In [4]:
# aanpassen van de data (NaN -> 0 en x -> 1)
dfProjectTasks = dfProjectTasks.fillna(0)
dfAdditionalTasks = dfAdditionalTasks.fillna(0)
dfCrew = dfCrew.fillna(0)
dfRooms = dfRooms.fillna(1)

dfProjectTasks.replace("x", 1, inplace=True)
dfAdditionalTasks.replace("x", 1, inplace=True)
dfCrew.replace("x", 1, inplace=True)


**Belangrijke info voor het plannen**
- De ingehuurde mensen hebben skill 1 en 8 
- kort inhuren is maximaal 60 uur
- het bedrijf is van 9-18 open
- een crewlid kan niet meer dan 3 uur achter elkaar werken, daarna een half uur pauze
- een crewlid werkt nooit meer dan 7 uur op een dag
- kan een 'acquire skill' kan voor ieder crewlid worden gedaan, geen beperking in aantal 
- crewleden moet te samen aan de skills komen voor de opdracht.
- Mensen die skill één niet hebben, hoeven deze niet te leren
- Een crewleden kunnen alleen aan eengesloten uren aan een project task werken. Dit mag niet worden doorbroken met een andere task. Met additional tasks mag dit wel. 
- iedereen moet tegelijk aan het project werken als ze op hetzelfde project staan.

Moet A volledig worden afgerond of maar 1 keer van het aantal frequenties? Moet volledig worden ingepland

We maken een soort kalender, startend op 6 januari 2020 en geen weekend dagen erin.
Hiervan maken we een dataframe met de volgende kolommen:
- datum
- een kolom met dataframes met de volgende punten:
    - tijd per halfuur
    - persoon
    - aantal uren te gaan
    - hierin staan de projecten met de kamer
- Of het een werkdag is (T/F)
- In welke sprint de dag zich bevindt.

In [5]:
# het maken van een dataframe als kalender << deze moet worden aangepast naar de info van hierboven

data = pd.date_range("2020-01-06", periods=4*10*7, freq='D')

columns = ['Datum', 'Sprint', 'Weeknummer', 'DagRooster', 'Werkdag']

dfKalenderCrew = pd.DataFrame(columns=columns)
dfKalenderCrew['Datum'] = data
dfKalenderCrew = dfKalenderCrew.fillna(0)
dfKalenderCrew.replace("0", False, inplace=True)

dfKalenderCrew['Werkdag'] = dfKalenderCrew['Datum'].apply(lambda Datum: Datum.isoweekday() in range (1,6))

In [6]:
dfKalenderCrew = dfKalenderCrew[dfKalenderCrew['Werkdag']]
dfKalenderCrew = dfKalenderCrew.reset_index(drop=True)

In [7]:
dfKalenderRoom = pickle.loads(pickle.dumps(dfKalenderCrew)) # het maken van een kalender van de kamers, tot dit punt waren de df's hetzelfde

In [8]:
# een paar standaard variabele om de code leesbaar te maken.
rangeSkills = range(1,14)
rangeWorkdays = range(14,19)
rangeRooms = range(14,20)

functie waarbij de lijst van ReqSkill moet worden ingevuld van de Task en de skills van een persoon. De functie geeft een boolean terug of deze taak door deze persoon kan worden gedaan.

voorbeeld: <br>
OpdrachtUitvoeren(dfProjectTasks.iloc[0,rangeSkills],dfCrew.iloc[0,rangeSkills]) 
<br>uitkomst: False

In [9]:
# def OpdrachtUitvoerenPersoonBool(eisen, persoon):
#     for i in rangeSkills:
#         result = (int(eisen[i-1]) <= int(persoon[i-1]))
#         if result == False:
#             break
#     return result

de functie hieronder voegt de skills van personen samen en geeft de gezamelijke skills terug en het surplus.

voorbeeld: PersonenSamenvoegen(dfCrew.iloc[0,rangeSkills],dfCrew.iloc[2,rangeSkills])
<br>
uitkomst: [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]

In [10]:
# def PersonenSamenvoegen(persoon1, persoon2 = [0]*13, persoon3 = [0]*13, persoon4 = [0]*13, persoon5 = [0]*13, persoon6 = [0]*13):
#     combinedSkill = []
#     surplus = 0 
#     for i in rangeSkills:
#         combinedSkill.append(int(1) if persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] > 0 else int(0))
#         surplus = surplus + (persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] -1 
#                          if persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] > 0 else int(0))
#     combinedSkill.append(surplus)
#     return combinedSkill

In [11]:
# het selecteren van alleen de tasks van sprint 1 en het dataframe maken met de columnnames en het resetten van de index
dfProjectTasks = dfProjectTasks[dfProjectTasks['Task'] != 0]
dfSprint1 = pd.DataFrame(dfProjectTasks[dfProjectTasks['Task'].str.contains('S1')], 
                         columns= list(dfProjectTasks)).reset_index(drop=True)


In [12]:
# het maken van de kolom crew en het aanpassen van het typen naar object zodat er een list of array kan worden toegevoegd aan die kolom.
dfSprint1.loc[:,'Crew']= [20]*len(dfSprint1)
dfSprint1['Crew'] = dfSprint1['Crew'].astype(object)

In [13]:
# # het verzamelen van data van alle gezamelijke skills in staan tot aan 7 personen

# dfCrewCombined = []
# for i in range(0, len(dfCrew)): 
#     t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills])
#     t.append(i)
#     t.append(20)
#     t.append(20)
#     t.append(20)
#     t.append(20)
#     t.append(20)
#     t.append(1)
    
#     dfCrewCombined.append(t)
    
#     for j in range(0, len(dfCrew)): # combinaties met twee mensen
#         if i==j:
#             break
#         t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills])
#         t.append(i)
#         t.append(j)
#         t.append(20)
#         t.append(20)
#         t.append(20)
#         t.append(20)
#         t.append(2)
        
#         dfCrewCombined.append(t)

#         for k in range(0, len(dfCrew)):
#             if i==k or k==j:
#                 break
        
#             t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills], dfCrew.iloc[k,rangeSkills])
#             t.append(i)
#             t.append(j)
#             t.append(k)
#             t.append(20)
#             t.append(20)
#             t.append(20)
#             t.append(3)
            
#             dfCrewCombined.append(t)
            
#             for l in range(0, len(dfCrew)):
#                 if l == k or l == i or l == j :
#                     break
#                 t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
#                                         dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills])
#                 t.append(i)
#                 t.append(j)
#                 t.append(k)
#                 t.append(l)
#                 t.append(20)
#                 t.append(20)
#                 t.append(4)
#                 dfCrewCombined.append(t)

#                 for m in range(0, len(dfCrew)):
#                     if m==l or m==k or m==j or m==i:
#                         break
#                     t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
#                                             dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills], 
#                                             dfCrew.iloc[m,rangeSkills])
#                     t.append(i)
#                     t.append(j)
#                     t.append(k)
#                     t.append(l)
#                     t.append(m)
#                     t.append(20)
#                     t.append(5)
#                     dfCrewCombined.append(t)
                    
#                     for n in range(0, len(dfCrew)):
#                         if n==m or n==l or n==k or n==j or n==i:
#                             break
#                         t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
#                                                 dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills], 
#                                                 dfCrew.iloc[m,rangeSkills], dfCrew.iloc[n,rangeSkills])
#                         t.append(i)
#                         t.append(j)
#                         t.append(k)
#                         t.append(l)
#                         t.append(m)
#                         t.append(n)
#                         t.append(6)
#                         dfCrewCombined.append(t)

het dataframe dfCrewCombined is een dataframe met alle skills van de medewerkers gezamelijk. Zodra er bij een van de personen '20' staat, betekent dit dat er geen persoon voor die plek is toegevoegd. in het dataframe staan combinaties met twee personen tot zeven ('vo) personen.


In [14]:
# # het maken van het dataframe dfCrewCombined en het wegschrijven in csv.
# ar = np.array(dfCrewCombined, dtype=np.int16) 
# dfCrewCombined = pd.DataFrame(ar, columns= ['Skill1', 'Skill2', 'Skill3', 'Skill4', 'Skill5', 'Skill6', 'Skill7', 'Skill8', 
#                                 'Skill9', 'Skill10', 'Skill11', 'Skill12', 'Skill13', 'Surplus', 'persoon1', 'persoon2', 
#                                 'persoon3', 'persoon4', 'persoon5', 'persoon6', 'Aantalpersonen'])

In [15]:
# # creeren van de kolom 'sum' zodat we weten hoeveel skill de crew gecombineerd heeft
# dfCrewCombined['sum'] = dfCrewCombined.iloc[:,0:13].sum(axis = 1) 

# # sorteren op de kolommen sum en surplus (overschot aan skills) zodat er zo min mogelijk overbodige skills worden gebruikt 
# dfCrewCombined = dfCrewCombined.sort_values(['sum', 'Surplus'], ascending= [True, True])
# dfCrewCombined = dfCrewCombined.reset_index(drop=True)

# dfCrewCombined.to_csv('CrewCombined.csv', encoding='utf-8', index=False) # schrijven naar csv

In [ ]:
CrewCombined.MakeCrewCombined(dfCrew)

In [ ]:
dfCrewCombined = pd.read_csv('CrewCombined.csv') # lezen van csv zodra die in wdir staat.

In [ ]:
class MogelijkeCombinaties:
    def __init__(self, dfMogelijkheden, aantalMogelijkheden):
        self.dfMogelijkheden = dfMogelijkheden
        self.aantalMogelijkheden = aantalMogelijkheden

In [ ]:
class Rooster:
    def __init__(self, dfRooster):
        self.dfRooster = dfRooster

In [ ]:
def Lijstmogelijkheden(crew, duration): # lijst met mogelijke data wordt teruggegeven zodra de lijn van de crew en de duur van de task wordt ingevuld
    lijstmogelijkheden = []
    for dag in werkdagen:
        if crew == dag.crew:
            if dag.urenTeGaan >= duration :
                lijstmogelijkheden.append(dag)
    if len(lijstmogelijkheden) == 0:
        lijstmogelijkheden.append(RoosterDag("geen medewerker",datetime.date(2100,1,1)))
                                  
    return lijstmogelijkheden

In [ ]:
dfSprint1.loc[:,'Mogelijkheden']= [20]*len(dfSprint1)
dfSprint1.loc[:,'AantalMogelijkheden']= [20]*len(dfSprint1)
dfSprint1['Mogelijkheden'] = dfSprint1['Mogelijkheden'].astype(object)
dfSprint1['AantalMogelijkheden'] = dfSprint1['AantalMogelijkheden'].astype(int)
dfSprint1['MaxReqCrew'].replace(0, 6, inplace=True)


In [ ]:
def ZoekKamers(eisenKamers):
    lijstkamers = []
    for i in range(0, len(eisenKamers)):
        if list(eisenKamers)[i] == 1:
            if ("****" in list(dfSprint1)[14+i]):
                lijstkamers.append("assembly A")
                lijstkamers.append("assembly B")
            if ("***" in list(dfSprint1)[14+i] and not "****" in list(dfSprint1)[14+i]):
                lijstkamers.append("workshop A")
                lijstkamers.append("workshop B")
                lijstkamers.append("workshop C")
            if ("**" in list(dfSprint1)[14+i] and not "***" in list(dfSprint1)[14+i]):
                lijstkamers.append("workspace A")
                lijstkamers.append("workspace B")
                lijstkamers.append("workspace C")
                
            elif ("4" in list(dfSprint1)[14+i] or "5" in list(dfSprint1)[14+i]):
                lijstkamers.append("office A1")
                lijstkamers.append("office A2")
                lijstkamers.append("office A3")
                lijstkamers.append("office A4")
                lijstkamers.append("office A5")
                lijstkamers.append("office B1")
                lijstkamers.append("office B2")
                lijstkamers.append("office B3")
                lijstkamers.append("office B4")
                lijstkamers.append("office C1")
                lijstkamers.append("office C2")
                lijstkamers.append("office C3")
                lijstkamers.append("office C4")
                
            elif ("technical" in list(dfSprint1)[14+i]):
                lijstkamers.append("technical A")
                lijstkamers.append("technical B")
                
    return lijstkamers

In [ ]:
def ZoekOpdrachten(project, MinMaxPersonen):
    crew = []
    for i in range(0, len(dfCrewCombined)):
        if (dfCrewCombined.Aantalpersonen[i] >= MinMaxPersonen[0] and dfCrewCombined.Aantalpersonen[i] <= MinMaxPersonen[1]):
            if (Hulpfuncties.OpdrachtUitvoerenPersoonBool(project, dfCrewCombined.iloc[i,0:13])):
                crew.append(list(dfCrewCombined.iloc[i,14:20]))
    return crew
                        

In [ ]:
dfSprint1['Voltooid'] = False
dfSprint1['Eis'] = ""
dfSprint1['Eis']=dfSprint1['Task'].apply(lambda Task: Task[:-1]+'A' if Task.endswith('B') else 'no')
dfSprint1['Moment Voltooid']= ""
dfSprint1['Dag Voltooid'] = ""

In [ ]:
# for i in range(0,len(dfSprint1)):
#     a = ZoekKamers(dfSprint1.iloc[i,14:20])
#     b = ZoekOpdrachten(dfSprint1.iloc[i,rangeSkills], dfSprint1.iloc[i,20:22]) 
#     c = pd.DataFrame(data = [[a]*len(b),b])
#     dfSprint1.iloc[i,26] = MogelijkeCombinaties(c.T, len(a) * len(b))
#     dfSprint1.iloc[i,27] = (dfSprint1["Frequency"][i]*dfSprint1["MinReqCrew"][i]) # >> is een optie, hoeft op zich niet, misschien wel makkelijker

In [ ]:
# # het schrijven van dfSprint naar een pickle bestand
# filehandler = open('dfSprint.obj', 'wb') 
# pickle.dump(dfSprint1, filehandler)

In [ ]:
# het lezen van dfSprint1, met meerdere keren runnen een heel stuk sneller
filehandler = open('dfSprint.obj', 'rb') 
dfSprint1 = pickle.load(filehandler)

In [ ]:
dfSprint1 = dfSprint1.sort_values(['AantalMogelijkheden'], ascending= [False])
dfSprint1 = dfSprint1.reset_index(drop=True)

In [ ]:
Eisen = dfSprint1.Eis.unique().tolist()
Eisen.remove('no')

dfSprint1 = (dfSprint1.loc[dfSprint1['Task'].isin(Eisen)].append(dfSprint1.loc[~dfSprint1['Task'].isin(Eisen)])).reset_index(drop=True)

In [ ]:
# het maken van een dataframe als kalender << deze moet worden aangepast naar de info van hierboven
tijd = pd.date_range("09:00", "17:30", freq="30min").time
tijd = np.delete(tijd, 13, axis=0)
tijd = np.delete(tijd, 6, axis=0)

columns = np.array(dfCrew['Crew members'])

dfRooster = pd.DataFrame(columns=columns)
dfRooster['Tijd'] = tijd
dfRooster = dfRooster.fillna(0)
dfRooster.replace("0", False, inplace=True)

# volgorde veranderen
cols = dfRooster.columns.tolist()
cols = cols[-1:] + cols[:-1]
dfRooster = dfRooster[cols]

In [ ]:
lijstkamers = dfRooms['Room'].iloc[0:10].tolist()

lijstkamers.append("office A1")
lijstkamers.append("office A2")
lijstkamers.append("office A3")
lijstkamers.append("office A4")
lijstkamers.append("office A5")
lijstkamers.append("office B1")
lijstkamers.append("office B2")
lijstkamers.append("office B3")
lijstkamers.append("office B4")
lijstkamers.append("office C1")
lijstkamers.append("office C2")
lijstkamers.append("office C3")
lijstkamers.append("office C4")

dfRoosterKamer = pd.DataFrame(columns=lijstkamers)
dfRoosterKamer['Tijd'] = tijd
dfRoosterKamer = dfRoosterKamer.fillna(0)
dfRoosterKamer.replace("0", False, inplace=True)

# volgorde veranderen
cols = dfRoosterKamer.columns.tolist()
cols = cols[-1:] + cols[:-1]
dfRoosterKamer = dfRoosterKamer[cols]


In [ ]:
a = [7]*(len(dfRooster.iloc[1, :])-1)
a.insert(0,'urenTeGaan')

b = dfRooster.columns.tolist()
new_row = pd.DataFrame(a).T
new_row.columns = b

dfRooster = pd.concat([new_row, dfRooster]).reset_index(drop = True)
StandaardRooster = dfRooster

In [ ]:
# inplannen stand-up op dinsdagochtend
dfRoosterDinsdag = copy.deepcopy(dfRooster)
dfRoosterDinsdag.iloc[1:7,1:14] = "weekly stand-up"
dfRoosterDinsdag.iloc[0,1:14] = 4

dfRoosterKamerDinsdag = copy.deepcopy(dfRoosterKamer)
dfRoosterKamerDinsdag.loc[0:5, 'workshop A'] ="weekly stand-up"

In [ ]:
for i in range(0,5): # het maken van de lege roosters voor week 1
    if i== 1:
        rooster = copy.deepcopy(dfRoosterDinsdag)
        rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist() # het maken van de lijst urentegaan met de vermenigvuldiging van de lijst met dagen dat iedereen werkt en een lijst met alleen maar 7's, resultaat: 7 als iemand werkt, als iemand niet werkt, 0.
        dfKalenderCrew.DagRooster[i] = Rooster(rooster)
        dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamerDinsdag))
    else:
        rooster = copy.deepcopy(StandaardRooster)
        rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist() # het maken van de lijst urentegaan met de vermenigvuldiging van de lijst met dagen dat iedereen werkt en een lijst met alleen maar 7's, resultaat: 7 als iemand werkt, als iemand niet werkt, 0.
        dfKalenderCrew.DagRooster[i] = Rooster(rooster)
        dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamer))
    
for i in range(1,11): # het kopieren van de roosters van week 1 voor de rest van de weken in de sprint
    for j in range(0,5):
        dfKalenderCrew.DagRooster[i*5 + j] = copy.deepcopy(dfKalenderCrew.DagRooster[j])
        dfKalenderRoom.DagRooster[i*5 + j] = copy.deepcopy(dfKalenderRoom.DagRooster[j])

In [ ]:
def contains(sub, pri):
    M, N = len(pri), len(sub)
    i, LAST = 0, M-N+1
    while True:
        try:
            found = pri.index(sub[0], i, LAST) # find first elem in sub
        except ValueError:
            return False
        if pri[found:found+N] == sub:
            return [found, found+N-1]
        else:
            i = found+1

In [ ]:
def CombinatieLijst(lijst1, lijst2):
    lijstfinal = []
    if ((len(lijst1) == 16) & (len(lijst2) == 16)):
        for i in range(0,len(lijst1)):
            if((lijst1[i]==0) & (lijst2[i] == 0)):
                lijstfinal.append(0)
            else:
                lijstfinal.append(1)
        return lijstfinal
    else:
        Exception("Lengte van de lijst klopt niet")
        return
    

!TODO opdrachten over twee dagen

!TODO Opdracht volgorde veranderen. A moet eerst!

In [ ]:
def InplannenOpdracht(taskdf, dagIndex, dfKalenderCrew, dfKalenderRoom, crewA = []): 
# task moet een rij van het dataframe dfSprint1 zijn en een dag (als index) van een rij in dfKalenderCrew (verschil ten opzichten van 2020-1-6)
#     print("Aanroep functie InplannenOpdracht met dagindex: " + str(dagIndex))
        
#     print("Er wordt ene kopie gemaakt van dfKalenderCrew")
    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    
    for crewlijst in taskdf.Mogelijkheden.dfMogelijkheden.iloc[:,1].tolist():
#         print("De crewlijst die wordt uitgeprobeerd: " + str(crewlijst))
        a = list(set(crewlijst))
        try:
            a.remove(20)
        except:
            None
        if ((len(set(a) & set(crewA)) >= 1) | (len(crewA)==0)):
            lijstVrijeUren = [0]*16
            controle = True
            dagIndexKopie = dagIndex

            freq = 1
            while (freq <= int(taskdf.Frequency)) & (taskdf.Voltooid == False):
    #             print("bovenaan de while loop met: "+ taskdf.Task)
                if (taskdf.Eis != 'no') & (dagIndex == dagIndexKopie):
                    eis = dfSprint1[dfSprint1.Task == task.Eis].iloc[0,:]
                    lijstVrijeUren = [1]*int(eis["Moment Voltooid"]) + [0]*(16-int(eis["Moment Voltooid"]))
                for i in crewlijst:
    #                 print("Kijken naar i: " + str(i))
                    if (i == 20):
                        break
                    if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(taskdf.Duration)):
    #                     print("De vergeleken waardes: "+ str(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1])+  " "+ str(task.Duration) )
    #                     print("De controle van het aantal uur te gaan is False")
                        controle = False
                        break 
    #                 print("de datum " + str(dfKalenderCrew.iloc[dagIndexKopie,:].Datum) + " voor persoon " +str(i))
    #                 print("Het aantal uren te gaan voor persoon i " + str(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))
                    lijstVrijeUren = CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment
                duration = [0]*int(round(taskdf.Duration*2))

    #             print("De lijstvrije uren ziet er als uit: " + str(lijstVrijeUren))
                if (contains(duration, lijstVrijeUren) != False) & controle:
                    rijbegin , rijeind = contains(duration, lijstVrijeUren)

    #                 print("het begin en het einde van de rij: "+ str(rijbegin +1) +" "+ str(rijeind+1))

                    kamerGevonden = False
                    for i in taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0]:
                        if (contains(duration, dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[1:].tolist()) != False):
    #                         print("kamer gevonden: " + i)
                            dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[rijbegin:rijeind+1] = taskdf.Task + " freq: " + str(freq)
                            kamerGevonden = True
                            controle = True # later toegevoegd 14:50 - 5-11
                            break
                        elif ((i == taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0][-1])& (kamerGevonden == False)):
    #                         print("Geen kamer gevonden, controle False")
                            controle = False
                    if (controle == True):
                        for i in crewlijst:
                            if i == 20:
                                break

                            dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task + " freq: " + str(freq) # + " " + str(taskdf.Mogelijkheden.dfMogelijkheden.iloc[crewlijst,0].iloc[0])
                            dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(taskdf.Duration)

                        if (freq == int(taskdf.Frequency)):
    #                         print("Voltooid " + taskdf.Task)
                            dfSprint1.loc[dfSprint1['Task'] == taskdf.Task, 'Moment Voltooid'] = rijeind +1
                            dfSprint1.loc[dfSprint1['Task'] == taskdf.Task, 'Dag Voltooid'] = dagIndexKopie
                            dfSprint1.loc[dfSprint1['Task'] == taskdf.Task, 'Voltooid'] = True
                            dfSprint1.at[dfSprint1.index[dfSprint1['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst


                            return [dfKalenderCrew, dfKalenderRoom]

    #                     print("ingepland" + taskdf.Task + " " + str(freq))
                        freq += 1
                else:
    #                 print("dagindex wordt verhoogd naar: " + str(dagIndexKopie+1))
                    dagIndexKopie+=1

                    controle = True
                    lijstVrijeUren = [0]*16
                    if (dagIndexKopie == 5):
    #                     print("Een kopie wordt teruggezet van dfKalenderCrew")
                        dfKalenderCrew = pickle.loads(pickle.dumps(dfTempCrew))
                        dfKalenderRoom = pickle.loads(pickle.dumps(dfTempRoom))
                        break
    return [dfKalenderCrew, dfKalenderRoom]
    
    

In [ ]:
dfk = pickle.loads(pickle.dumps(dfKalenderCrew))
dfr = pickle.loads(pickle.dumps(dfKalenderRoom))
dfs = pickle.loads(pickle.dumps(dfSprint1))

In [ ]:
# dfKalenderCrew = pickle.loads(pickle.dumps(dfk))
# dfSprint1 = pickle.loads(pickle.dumps(dfs))

In [ ]:
for i in range(0,len(dfSprint1)):
    task = dfSprint1.iloc[i,:]
    dagIndex = 0
    # print(task.Task + " met: " + str(task.Voltooid))
    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    crewA = []
    if task.Eis != 'no':
        eis = dfSprint1[dfSprint1.Task == task.Eis].iloc[0,:]
        dfKalenderCrew, dfKalenderRoom = InplannenOpdracht(eis,dagIndex, dfKalenderCrew, dfKalenderRoom)
        eis = dfSprint1[dfSprint1.Task == task.Eis].iloc[0,:]
        print(eis.Task)
        dagIndex = int(eis['Dag Voltooid'])
        crewA = eis.Crew
    dfKalenderCrew, dfKalenderRoom = InplannenOpdracht(task,dagIndex, dfKalenderCrew, dfKalenderRoom, crewA)
if dfSprint1.Voltooid.sum()< len(dfSprint1):
    dfSprint1 = dfSprint1.sort_values(['Voltooid', 'AantalMogelijkheden'], ascending= [True, False])
    dfSprint1 = (dfSprint1.loc[dfSprint1['Task'].isin(Eisen)].append(dfSprint1.loc[~dfSprint1['Task'].isin(Eisen)])).reset_index(drop=True)
    rows = dfSprint1.Task.tolist()
    
    dfKalenderCrew = pickle.loads(pickle.dumps(dfk))
    dfKalenderRoom = pickle.loads(pickle.dumps(dfr))
    dfSprint1 = pickle.loads(pickle.dumps(dfs))
    dfSprint1 = dfSprint1.set_index('Task')
    dfSprint1 = dfSprint1.reindex(index= rows)
    dfSprint1 = dfSprint1.reset_index()
    
    for i in range(0,len(dfSprint1)):
        task = dfSprint1.iloc[i,:]
        dagIndex = 0
        # print(task.Task + " met: " + str(task.Voltooid))
        dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
        dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
        crewA = []
        if task.Eis != 'no':
            eis = dfSprint1[dfSprint1.Task == task.Eis].iloc[0,:]
            dfKalenderCrew, dfKalenderRoom = InplannenOpdracht(eis,dagIndex, dfKalenderCrew, dfKalenderRoom)
            eis = dfSprint1[dfSprint1.Task == task.Eis].iloc[0,:]
            print(eis.Task)
            dagIndex = int(eis['Dag Voltooid'])
            crewA = eis.Crew
        dfKalenderCrew, dfKalenderRoom = InplannenOpdracht(task,dagIndex, dfKalenderCrew, dfKalenderRoom, crewA)

In [ ]:
dfSprint1.iloc[:,[0,20,21,22,23,27,28,29,30,31]]

In [ ]:
dfKalenderCrew.iloc[0,:].DagRooster.dfRooster

In [ ]:
dfKalenderCrew.iloc[1,:].DagRooster.dfRooster

In [ ]:
dfKalenderCrew.iloc[2,:].DagRooster.dfRooster

In [ ]:
dfKalenderCrew.iloc[3,:].DagRooster.dfRooster

In [ ]:
dfKalenderCrew.iloc[4,:].DagRooster.dfRooster

In [ ]:
def EvaluateProgram(dfTasks, dfRoosterVoor, dfRoosterNa):
    urenTeGaan, totaal, som, count = [0,0,0,0]
    for i in range(0,5): # deze kan worden verhoogt zodra er meer dagen worden ingevuld.
        urenTeGaan += sum(dfRoosterVoor.iloc[i,:].DagRooster.dfRooster.iloc[0,:].tolist()[1:])
        totaal += sum(dfRoosterNa.iloc[i,:].DagRooster.dfRooster.iloc[0,:].tolist()[1:])
    for i in range(0, len(dfTasks)):
        som += dfTasks.loc[i, 'MinReqCrew'] * dfTasks.loc[i,'Frequency'] * dfTasks.loc[i,'Duration']
        if (dfTasks.Voltooid[i] == False):
            count += 1
    print("Het aantal uur dat nog over is: " + str(totaal))
    print("Van het totaal aantal uren: " + str(urenTeGaan))
    print("Het aantal uur dat moest worden ingepland: " + str(som))
    print("Het aantal opdrachten dat niet zijn ingepland: " + str(count))

In [ ]:
EvaluateProgram(dfSprint1,dfk,dfKalenderCrew)

In [ ]:
dfSprint1

In [ ]:
dfKalenderRoom.iloc[0,:].DagRooster.dfRooster

!TODO 99% betrouwbaarheidsinterval

Het bedrijf werkt in sprints. Om precies te zijn: 4 sprints van elk 10 weken. De weken
2 t/m 8 van elke sprint zijn de zogeheten projectweken

In [ ]:
# het kopieren van de roosters. Deze zijn voor dit gedeelte gelijk in week 2 tot en met 8. 
# Hierna moeten de additional tasks worden ingepland.

for i in range(1,8):
    for j in range(0,5):
        dfKalenderRoom.iloc[i*5 + j,:].DagRooster.dfRooster = pickle.loads(pickle.dumps(dfKalenderRoom.iloc[j,:].DagRooster.dfRooster))
        dfKalenderCrew.iloc[i*5 + j,:].DagRooster.dfRooster = pickle.loads(pickle.dumps(dfKalenderCrew.iloc[j,:].DagRooster.dfRooster))
        
for i in range(0,5): # leeg maken van de eerste week.
    if i== 1:
        rooster = copy.deepcopy(dfRoosterDinsdag)
        rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist() # het maken van de lijst urentegaan met de vermenigvuldiging van de lijst met dagen dat iedereen werkt en een lijst met alleen maar 7's, resultaat: 7 als iemand werkt, als iemand niet werkt, 0.
        dfKalenderCrew.DagRooster[i] = Rooster(rooster)
        dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamerDinsdag))
    else:
        rooster = copy.deepcopy(StandaardRooster)
        rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist() # het maken van de lijst urentegaan met de vermenigvuldiging van de lijst met dagen dat iedereen werkt en een lijst met alleen maar 7's, resultaat: 7 als iemand werkt, als iemand niet werkt, 0.
        dfKalenderCrew.DagRooster[i] = Rooster(rooster)
        dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamer))

!TODO de mensen die de prep uitvoeren, moeten ook de projecttasks en post doen.

Wanneer kan er met de prep check-up worden begonnen en wanneer kan de check-up worden gedaan?

In [ ]:
dfk = pickle.loads(pickle.dumps(dfKalenderCrew))
dfs = pickle.loads(pickle.dumps(dfSprint1))
dfr = pickle.loads(pickle.dumps(dfKalenderRoom))


In [ ]:
dfAdditionalTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Additional tasks')

In [ ]:
dfAdditionalTasks = dfAdditionalTasks.fillna(0)
dfAdditionalTasks.replace("x", 1, inplace=True)

!TODO kiezen van de crew director

In [ ]:
dfAdditionalTasks = dfAdditionalTasks[dfAdditionalTasks['Task'] != 0].reset_index(drop=True)
dfAdditionalTasks['aantal keer'].replace(0, 1, inplace=True)
dfSprint1Add = pd.DataFrame(dfAdditionalTasks[dfAdditionalTasks['Task'].str.contains('S1')]).reset_index(drop=True)
dfSprint1Addprep = pd.DataFrame(dfSprint1Add[dfSprint1Add['Task'].str.contains('prep')]).reset_index(drop=True)
dfSprint1Addpost = pd.DataFrame(dfSprint1Add[dfSprint1Add['Task'].str.contains('post')]).reset_index(drop=True)


In [ ]:
dfAdditionalTasks

In [ ]:
def somOverigeUren(crewlijst, weeknummers):
    resultaat = pd.DataFrame(columns=['crew', 'som'])
    for i in crewlijst:
        som = 0 
        if (i == 20):
            break
        for weeknummer in weeknummers:
            for j in range(0 + 5*(weeknummer-1),5*weeknummer):
                som += dfKalenderCrew.iloc[j,:].DagRooster.dfRooster.iloc[0,i+1] # want kolom tijd wordt ook meegenomen
        resultaat = resultaat.append({'crew':i,'som': int(som)}, ignore_index=True)
    print(resultaat)
    if(type(resultaat) == 'NoneType'):
        Exception("Nonetype")
    return resultaat.sort_values(['som'], ascending= [False])

In [ ]:
def InplannenAddPrep(taskdf, dfKalenderCrew, dfKalenderRoom, prep = True): 

    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    
    # crew zelf maken
    crew = []
    dagIndex = 0
    if (taskdf.AdditionalReq == 0):
        
        task = taskdf.Task[5:]
        crewlijst = dfSprint1[dfSprint1.Task == task].iloc[0,:].Crew
        minCrew = taskdf['People.1']
        
        crewlijst = somOverigeUren(crewlijst, [1])['crew'].iloc[:minCrew].to_list()
        crewlijst.extend([20]*(6-minCrew))
        print(str(crewlijst) + "%")        
    elif ((len(taskdf.AdditionalReq) > 5) & (len(taskdf.AdditionalReq) < 8)):
        crewlijst = dfSprint1[dfSprint1.Task == taskdf.AdditionalReq].iloc[0,:].Crew
    elif (len(taskdf.AdditionalReq) > 8 ):
        taskA = taskdf.AdditionalReq[0:7]
        taskB = taskdf.AdditionalReq[-7:]
        taskACrew = dfSprint1[dfSprint1.Task == taskA].iloc[0,:].Crew
        taskBCrew = dfSprint1[dfSprint1.Task == taskB].iloc[0,:].Crew
        crewlijst = [20]*6
        b = list(set(taskACrew)&set(taskBCrew))
        try:
            b.remove(20)
        except:
            None
        crewlijst[0] = b[0]
        
    print(crewlijst)
    a = list(set(crewlijst))
    try:
        a.remove(20)
    except:
        None
    aantalCrew = len(a)
    print(int(aantalCrew))
    print(int(taskdf['People.1']))
    if (int(taskdf['People.1'])==int(aantalCrew)):
        print("nu 1")
        kolom = 1
    elif (int(taskdf['People.2'])==int(aantalCrew)):
        kolom = 2
    elif (int(taskdf['People.3'])==int(aantalCrew)):
        kolom = 3
    
    lijstVrijeUren = [0]*16
    controle = True
    dagIndexKopie = dagIndex
    halfuur = 0
    
    while halfuur < int(round(taskdf[str("DurationExp." + str(kolom))]*2)):
        for i in crewlijst:
            if (i == 20):
                break
            if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(0.5)):
                controle = False
                break 
            lijstVrijeUren = CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment
        
        if (int(round(taskdf[str("DurationExp." + str(kolom))]*2)) == 0):
            dfSprint1Addprep.loc[dfSprint1Addprep['Task'] == taskdf.Task, 'Voltooid'] = True
            break


        if (contains([0], lijstVrijeUren) != False) & controle:
            rijbegin , rijeind = contains([0], lijstVrijeUren)

            kamerGevonden = False

            kamers = ZoekKamers(taskdf.iloc[14:20])

            for kamer in kamers:
                if (contains([0], dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[1:].tolist()) != False):

                    dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[rijbegin:rijeind+1] = taskdf.Task
                    kamerGevonden = True
                    controle = True
                    break
                elif ((kamer == kamers[-1]) & (kamerGevonden == False)):                       
                    controle = False

            if (controle == True):
                for i in crewlijst:
                    if i == 20:
                        break

                    dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task 
                    dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(0.5)
#                     dfSprint1Addprep.loc[dfSprint1Addprep['Task'] == taskdf.Task, 'Voltooid'] = True
                    dfSprint1Addprep.at[dfSprint1Addprep.index[dfSprint1Addprep['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst
                    halfuur +=1
#                     return [dfKalenderCrew, dfKalenderRoom]
                    if (halfuur == (int(round(taskdf[str("DurationExp." + str(kolom))]*2))-1)):
                        dfSprint1Addprep.loc[dfSprint1Addprep['Task'] == taskdf.Task, 'Voltooid'] = True
                        return [dfKalenderCrew, dfKalenderRoom]

        else:

            dagIndexKopie+=1

            controle = True
            lijstVrijeUren = [0]*16
            if (dagIndexKopie == 5):
                dfKalenderCrew = pickle.loads(pickle.dumps(dfTempCrew))
                dfKalenderRoom = pickle.loads(pickle.dumps(dfTempRoom))
                break

    print("Niet ingepland")
    return [dfKalenderCrew, dfKalenderRoom]

In [ ]:
def InplannenAddPost(taskdf, dfKalenderCrew, dfKalenderRoom, prep = True): 

    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    
    # crew zelf maken
    crew = []
    dagIndex = 40
    if (taskdf.AdditionalReq == 0):
        
        task = taskdf.Task[5:]
        crewlijst = dfSprint1[dfSprint1.Task == task].iloc[0,:].Crew
        minCrew = taskdf['People.1']
        
        crewlijst = somOverigeUren(crewlijst, [9,10])['crew'].iloc[:minCrew].to_list()
        crewlijst.extend([20]*(6-minCrew))
        print(str(crewlijst) + "%")        
    elif ((len(taskdf.AdditionalReq) > 5) & (len(taskdf.AdditionalReq) < 8)):
        crewlijst = dfSprint1[dfSprint1.Task == taskdf.AdditionalReq].iloc[0,:].Crew
    elif (len(taskdf.AdditionalReq) > 8 ):
        taskA = taskdf.AdditionalReq[0:7]
        taskB = taskdf.AdditionalReq[-7:]
        taskACrew = dfSprint1[dfSprint1.Task == taskA].iloc[0,:].Crew
        taskBCrew = dfSprint1[dfSprint1.Task == taskB].iloc[0,:].Crew
        crewlijst = [20]*6
        b = list(set(taskACrew)&set(taskBCrew))
        try:
            b.remove(20)
        except:
            None
        crewlijst[0] = b[0]
        
    print(crewlijst)
    a = list(set(crewlijst))
    kolom = 1
    try:
        a.remove(20)
    except:
        None
    aantalCrew = len(a)
    if (int(taskdf['People.1'])==int(aantalCrew)):
        print("nu 1")
        kolom = 1
    elif (int(taskdf['People.2'])==int(aantalCrew)):
        kolom = 2
    elif (int(taskdf['People.3'])==int(aantalCrew)):
        kolom = 3
    
    lijstVrijeUren = [0]*16
    controle = True
    dagIndexKopie = dagIndex
    halfuur = 0
    
    while halfuur < int(round(taskdf[str("DurationExp." + str(kolom))]*2)):
        for i in crewlijst:
            if (i == 20):
                break
            if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(0.5)):
                print("Controle wordt op False gezet")
                controle = False
                break 
            lijstVrijeUren = CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment
            print("lijst vrije uren: " + str(lijstVrijeUren))
        if (int(round(taskdf[str("DurationExp." + str(kolom))]*2)) == 0):
            print("duration is nul")
            dfSprint1Addpost.loc[dfSprint1Addpost['Task'] == taskdf.Task, 'Voltooid'] = True
            break


        if (contains([0], lijstVrijeUren) != False) & controle:
            print("rijbegin en rijeind berekenen")
            rijbegin , rijeind = contains([0], lijstVrijeUren)

            kamerGevonden = False

            kamers = ZoekKamers(taskdf.iloc[14:20])

            for kamer in kamers:
                print("checken voor kamer: " + str(kamer))
                if (contains([0], dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[1:].tolist()) != False):

                    dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[rijbegin:rijeind+1] = taskdf.Task
                    kamerGevonden = True
                    controle = True
                    print("kamer gevonden, controle true")
                    break
                elif ((kamer == kamers[-1]) & (kamerGevonden == False)):   
                    print("geen kamer gevonden en door alle kamers heen")
                    controle = False

            if (controle == True):
                for i in crewlijst:
                    if i == 20:
                        break
                    print("Het inplannen van een halfuur")
                    dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task 
                    dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(0.5)
                    dfSprint1Addpost.at[dfSprint1Addpost.index[dfSprint1Addpost['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst
                    halfuur += 1
                    
                    if (halfuur == (int(round(taskdf[str("DurationExp." + str(kolom))]*2))-1)):
                        print("Gelukt, ingepland")
                        dfSprint1Addpost.loc[dfSprint1Addpost['Task'] == taskdf.Task, 'Voltooid'] = True
                        return [dfKalenderCrew, dfKalenderRoom]

        else:

            dagIndexKopie+=1
            print("dagindex kopie is verhoogd, " + str(dagIndexKopie))
            controle = True
            lijstVrijeUren = [0]*16
            if (dagIndexKopie == 50):
                print("dagindex is 50, "+ str(dagIndexKopie))
                dfKalenderCrew = pickle.loads(pickle.dumps(dfTempCrew))
                dfKalenderRoom = pickle.loads(pickle.dumps(dfTempRoom))
                break

    print("Niet ingepland")
    return [dfKalenderCrew, dfKalenderRoom]

In [ ]:
dfSprint1Addprep['Voltooid'] = False
dfSprint1Addprep.loc[:,'Crew']= [20]*len(dfSprint1Addprep)
dfSprint1Addprep['Crew'] = dfSprint1Addprep['Crew'].astype(object)
for i in range(0,len(dfSprint1Addprep)):
    task = dfSprint1Addprep.iloc[i,:]

    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    print(task.Task)
    dfKalenderCrew, dfKalenderRoom = InplannenAddPrep(task, dfKalenderCrew, dfKalenderRoom)

In [ ]:
dfSprint1Addpost['Voltooid'] = False
dfSprint1Addpost.loc[:,'Crew']= [20]*len(dfSprint1Addpost)
dfSprint1Addpost['Crew'] = dfSprint1Addpost['Crew'].astype(object)
for i in range(0,len(dfSprint1Addpost)):
    task = dfSprint1Addpost.iloc[i,:]

    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    print(task.Task)
    dfKalenderCrew, dfKalenderRoom = InplannenAddPost(task, dfKalenderCrew, dfKalenderRoom)

In [ ]:
dfKalenderCrew.iloc[4,:].DagRooster.dfRooster

In [ ]:
dfKalenderCrew.iloc[44,:].DagRooster.dfRooster

In [ ]:
dfSprint1Addprep

In [ ]:
dfSprint1